# Forest Fire Burn Area Prediction
Written by: Marshall Mykietyshyn

Data hosted on [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Forest+Fires).

The purpose of this notebook is to predict the forest fire burn area. This is a popular regression dataset found on both Kaggle and the UCI Machine Learning Repository. The two data sets are identical, but the one used here was downloaded from UCI. The forest fire weather index (FWI) system features FFMC, DMC, DC and ISI are described [here](https://cwfis.cfs.nrcan.gc.ca/background/summary/fwi). The target variable is burn area, which can be predicted by a set of feature variables listed below. The data to be used is already pre-processed by checking for missing values, outliers and converting the categorical features numerical values. However, there are unbalanced features that must be addressed, the target has nearly a 50% proportion of zeros, and the features are generally not Gaussian distributed. 

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: 0 to 11
4. day - day of the week: 0 to 6
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3
7. DC - DC index from the FWI system: 7.9 to 860.6
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40
12. rain - outside rain in mm/m2 : 0.0 to 6.4
13. area - the burned area of the forest (in ha): 0.00 to 1090.84

Notes:
- First, try predicting burn area directly
    - Resample to account for feature imbalances
    - Eliminate day category as it is likely just noise
    - Maybe eliminate rain due to high number of zeros
- Second, split the problem into classification (predict whether a fire occurs) and regression (if there is a fire, predict the burn area)
    - Can represent the results as a joint probability see [here](https://datascience.stackexchange.com/questions/25576/best-method-to-deal-with-too-many-zeroes-in-regression-problem)
- Look at the effects of whitening to improve model performance
- Maximize predictive capabilities of model
    - Do some research to determine which model is best suited to the task